<a href="https://colab.research.google.com/github/Kerriea-star/Advanced/blob/master/Getting_Started_with_KerasNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

KerasNLP is a natural language processing library that supports users through their entire development cycle. Our workflows are built from modular components that have state-of-the-art preset weights and architectures when used out-of-the-box and are easily customizable when more control is needed.

This library is an extension of the core Keras API; all high-level modules are Layers or Models.

KerasNLP uses the **Keras Core** library to work with any of TensorFlow, Pytorch and Jax. In the guide below, we will use the `jax` backend for training our models, and `tf.data` for efficiently running our input preprocessing. But feel free to mix things up! This guide runs in TensorFlow or PyTorch backends with zero changes, simply update the `KERAS_BACKEND` below.


Search Keras documentation...

» Developer guides / KerasNLP / Getting Started with KerasNLP
Getting Started with KerasNLP
Author: Jonathan Bischof
Date created: 2022/12/15
Last modified: 2023/07/01
Description: An introduction to the KerasNLP API.

 View in Colab • GitHub source

Introduction
KerasNLP is a natural language processing library that supports users through their entire development cycle. Our workflows are built from modular components that have state-of-the-art preset weights and architectures when used out-of-the-box and are easily customizable when more control is needed.

This library is an extension of the core Keras API; all high-level modules are Layers or Models. If you are familiar with Keras, congratulations! You already understand most of KerasNLP.

KerasNLP uses the Keras Core library to work with any of TensorFlow, Pytorch and Jax. In the guide below, we will use the jax backend for training our models, and tf.data for efficiently running our input preprocessing. But feel free to mix things up! This guide runs in TensorFlow or PyTorch backends with zero changes, simply update the KERAS_BACKEND below.

This guide demonstrates our modular approach using a sentiment analysis example at six levels of complexity:

*   Inference with a pretrained classifier

*   Fine tuning a pretrained backbone
*   Fine tuning with user-controlled preprocessing
*   Fine tuning a custom model
*   Pretraining a backbone model
*   LBuild and train your own transformer from scratch

Throughout our guide, we use Professor Keras, the official Keras mascot, as a visual
reference for the complexity of the material:

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_evolution.png" alt="drawing" height="250"/>

In [1]:
!pip install -q --upgrade keras-nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.9 MB/s eta 0:00:00


In [2]:
import os

os.environ["KERAS_BACKEND"] = "jax" # or "tensorflow" or "torch"

import keras_nlp
import keras_core as keras

Using JAX backend.


## API quickstart

Our highest level API is `keras_nlp.models`. These symbols cover the complete user
journey of converting strings to tokens, tokens to dense features, and dense features to
task-specific output. For each `XX` architecture (e.g., `Bert`), we offer the following
modules:

* **Tokenizer**: `keras_nlp.models.XXTokenizer`
  * **What it does**: Converts strings to sequences of token ids.
  * **Why it's important**: The raw bytes of a string are too high dimensional to be useful
    features so we first map them to a small number of tokens, for example `"The quick brown
    fox"` to `["the", "qu", "##ick", "br", "##own", "fox"]`.
  * **Inherits from**: `keras.layers.Layer`.
* **Preprocessor**: `keras_nlp.models.XXPreprocessor`
  * **What it does**: Converts strings to a dictionary of preprocessed tensors consumed by
    the backbone, starting with tokenization.
  * **Why it's important**: Each model uses special tokens and extra tensors to understand
    the input such as delimiting input segments and identifying padding tokens. Padding each
    sequence to the same length improves computational efficiency.
  * **Has a**: `XXTokenizer`.
  * **Inherits from**: `keras.layers.Layer`.
* **Backbone**: `keras_nlp.models.XXBackbone`
  * **What it does**: Converts preprocessed tensors to dense features. *Does not handle
    strings; call the preprocessor first.*
  * **Why it's important**: The backbone distills the input tokens into dense features that
    can be used in downstream tasks. It is generally pretrained on a language modeling task
    using massive amounts of unlabeled data. Transferring this information to a new task is a
    major breakthrough in modern NLP.
  * **Inherits from**: `keras.Model`.
* **Task**: e.g., `keras_nlp.models.XXClassifier`
  * **What it does**: Converts strings to task-specific output (e.g., classification
    probabilities).
  * **Why it's important**: Task models combine string preprocessing and the backbone model
    with task-specific `Layers` to solve a problem such as sentence classification, token
    classification, or text generation. The additional `Layers` must be fine-tuned on labeled
    data.
  * **Has a**: `XXBackbone` and `XXPreprocessor`.
  * **Inherits from**: `keras.Model`.

Here is the modular hierarchy for `BertClassifier` (all relationships are compositional):

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/class_diagram.png" alt="drawing" height="300"/>

All modules can be used independently and have a `from_preset()` method in addition to
the standard constructor that instantiates the class with **preset** architecture and
weights (see examples below).

## Data
We will use a running example of sentiment analysis of IMDB movie reviews. In this task, we use the text to predict whether the review was positive (`label = 1`) or negative (`label = 0`).

We load the data using `keras.utils.text_dataset_from_directory`, which utilizes the powerful `tf.data.Dataset` format for examples.

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  20.8M      0  0:00:03  0:00:03 --:--:-- 20.8M


In [4]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size = BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size = BATCH_SIZE,
)

# Inspec first review
# Format is (review text tensor, label tensor)

print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'I always enjoy seeing movies that make you think, and don\'t just drip-feed the answers to their audience. "Revolver" is one of these films, and although many reviewers have stated that it is difficult to follow, with a bit of concentration and an open mind I got it. First time. True, it doesn\'t compare to other mind-mucks like "The Usual Suspects" or "Memento", but in its own right its an intelligent and thought-provoking film. <br /><br />Another thing I really liked about this film is how damn beautiful it is. Every scene, every camera angle seems to have been thought about for ages. If you see it you\'ll know what I mean.<br /><br />So, to conclude... watch it with an open mind and you may enjoy it. If not, well, no-one ever said "Revolver" is for everyone. And that\'s my 2 cents.'>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)


## Inference with a pretrained classifier

<img src="https://storage.googleapis.com/keras-nlp/getting_started_guide/prof_keras_beginner.png" alt="drawing" height="250"/>

The highest level module in KerasNLP is a **task**. A **task** is a `keras.Model`
consisting of a (generally pretrained) **backbone** model and task-specific layers.
Here's an example using `keras_nlp.models.BertClassifier`.

**Note**: Outputs are the logits per class (e.g., `[0, 0]` is 50% chance of positive). The output is
[negative, positive] for binary classification.

In [6]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

231508/231508 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step       


17596448/17596448 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[-1.5387032,  1.5418268]], dtype=float32)

All tasks have a `from_preset` method that constructs a `keras.Model` instance with preset preprocessing, architecture and weights. This means that we can pass raw strings in any format accepted by a `keras.Model` and get output specific to our task.

This particular preset is a `"bert_tiny_uncased_en"` backbone fine-tuned on `sst2`, another movie review sentiment analysis (this time from Rotten Tomatoes). We use the `tiny` architecture for demo purposes, but larger models are recommended for SoTA performance.

Let's evaluate our classifier on the IMDB dataset. You will note we don't need to call `keras.Model.compile` here. All task models like `BertClassifier` ship with compilation defaults, meaning we can just call `keras.Model.evaluate` directly. You can always call compile as normal to override these defaults (e.g. to add new metrics).

The output below is [loss, accuracy],

In [7]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 802s 512ms/step - loss: 0.4632 - sparse_categorical_accuracy: 0.7826


[0.46294263005256653, 0.783519983291626]